# Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below.

In [2]:
!conda install -c conda-forge folium --yes

import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans

print('OK')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    branca:          0.3.1-py_0        conda-forge
    folium:          

In [3]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
rp = urlopen(link).read().decode('utf-8')
page = BeautifulSoup(rp, 'html.parser')
table = page.body.table.tbody

In [4]:
#functions for getting cell and row data

def table_cell(i):
    cells = i.find_all('td')
    row = []
    
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
        
    return row

def table_row():    
    data = []  
    
    for tr in table.find_all('tr'):
        row = table_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data

In [5]:
data = table_row()
columns = ['Postcode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(data, columns=columns)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
#dropping the "Not Assigned" borough
df1 = df[df.Borough != 'Not assigned']
df1 = df1.sort_values(by=['Postcode','Borough'])

df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Port Union


In [8]:
df_postkod = df1['Postcode']
df_postkod.drop_duplicates(inplace=True)
df2 = pd.DataFrame(df_postkod)
df2['Borough'] = '';
df2['Neighbourhood'] = '';

df2.reset_index(inplace=True)
df2.drop('index', axis=1, inplace=True)
df1.reset_index(inplace=True)
df1.drop('index', axis=1, inplace=True)

for i in df2.index:
    for j in df1.index:
        if df2.iloc[i, 0] == df1.iloc[j, 0]:
            df2.iloc[i, 1] = df1.iloc[j, 1]
            df2.iloc[i, 2] = df2.iloc[i, 2] + ',' + df1.iloc[j, 2]
            
for i in df2.index:
    s = df2.iloc[i, 2]
    if s[0] == ',':
        s =s [1:]
    df2.iloc[i,2 ] = s
    
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
df2.shape

(103, 3)

In [12]:
df2['Latitude'] = '0';
df2['Longitude'] = '0';
koordinat = pd.read_csv('https://cocl.us/Geospatial_data')

In [13]:
for i in df2.index:
    for j in koordinat.index:
        if df2.iloc[i, 0] == koordinat.iloc[j, 0]:
            df2.iloc[i, 3] = koordinat.iloc[j, 1]
            df2.iloc[i, 4] = koordinat.iloc[j, 2]          
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

Choose the neighbourhoods that contain word " Downtown Toronto":

In [14]:
trt = df2[df2['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
trt.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.6796,-79.3775
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668,-79.3677
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
3,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3789


In [15]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The Coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The Coordinates of Toronto are 43.653963, -79.387207.


In [16]:
map_trt = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, label in zip(trt['Latitude'], trt['Longitude'], trt['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_trt)  
    
map_trt

In [18]:
CLIENT_ID = '1XBGFEWWTM0WAYL1MWAL3SR1KK2SRIVNMVJMPNOQJTISYQXV' # your Foursquare ID
CLIENT_SECRET = 'E4UFQMPZUT5KQXYN0IZOYQX2XGSNIEV2PRZPNS0CAAFJI2L1' # your Foursquare Secret
VERSION = '20190323' # Foursquare API version

Borrowing the function from the lab to get Top 100 venues in Downtown Toronto within a radius of 500m:

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.


In [20]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

venues = getNearbyVenues(names=trt['Neighbourhood'],
                                   latitudes=trt['Latitude'],
                                   longitudes=trt['Longitude']
                                  )

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Queen's Park


In [21]:
#checking the size of venues dataframe
venues.shape

(1302, 7)

In [22]:
#checking how many unique categories of venues are there
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 205 unique categories.


In [23]:
# one hot encoding
dt_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

dt_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
dt_onehot.shape

(1302, 205)

Gouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category:

In [25]:
dt_grouped = dt_onehot.groupby('Neighborhood').mean().reset_index()
dt_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.010000,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.018182,0.000000,0.000000,0.000000,0.00
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00
3,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,...,0.000000,0.000000,0.00,0.00,0.00,0.011628,0.000000,0.000000,0.011628,0.00
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.048780,0.000000,0.048780,0.012195,0.00
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00
7,Church and Wellesley,0.012195,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,...,0.012195,0.012195,0.00,0.00,0.00,0.000000,0.000000,0.012195,0.000000,0.00
8,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.00
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.010000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.00


In [26]:
dt_grouped.shape

(19, 205)

Printing out each neighborhood along with the top 5 most common venues in it:

In [27]:
num_top_venues = 5

for hood in dt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.07
1  Thai Restaurant  0.04
2             Café  0.04
3       Steakhouse  0.04
4   Cosmetics Shop  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1          Steakhouse  0.04
2        Cocktail Bar  0.04
3         Cheese Shop  0.04
4  Seafood Restaurant  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3     Boat or Ferry  0.06
4           Airport  0.06


----Cabbagetown,St. James Town----
         venue  freq
0         Café  0.07
1  Coffee Shop  0.07
2       Bakery  0.05
3   Restaurant  0.05
4  Pizza Place  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1                Café  0.05
2  Italian Restaurant  0.05
3      Sandwich Place  0.05
4        Burger Joint  0.03




In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 25
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dt_grouped['Neighborhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Burger Joint,Restaurant,Bar,Bakery,Asian Restaurant,...,Pizza Place,Vegetarian / Vegan Restaurant,Gym,American Restaurant,Gastropub,New American Restaurant,Mediterranean Restaurant,Electronics Store,Clothing Store,Modern European Restaurant
1,Berczy Park,Coffee Shop,Beer Bar,Cheese Shop,Seafood Restaurant,Café,Farmers Market,Cocktail Bar,Steakhouse,Bakery,...,Hotel,Restaurant,Bistro,Concert Hall,Breakfast Spot,Park,Diner,Italian Restaurant,Japanese Restaurant,Jazz Club
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boat or Ferry,Plane,Rental Car Location,Coffee Shop,Sculpture Garden,...,Donut Shop,Doner Restaurant,Dog Run,Eastern European Restaurant,Discount Store,Diner,Women's Store,Deli / Bodega,Dessert Shop,Department Store
3,"Cabbagetown,St. James Town",Coffee Shop,Café,Restaurant,Pub,Italian Restaurant,Bakery,Pizza Place,Caribbean Restaurant,Breakfast Spot,...,Beer Store,Convenience Store,Market,Liquor Store,Deli / Bodega,Diner,Jewelry Store,Japanese Restaurant,Indian Restaurant,Flower Shop
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Burger Joint,Juice Bar,Japanese Restaurant,Ice Cream Shop,Bar,...,Yoga Studio,Office,Miscellaneous Shop,Park,Poke Place,Portuguese Restaurant,Modern European Restaurant,Korean Restaurant,Metro Station,Comic Shop


In [30]:
kclusters = 8
dt_group_cluster = dt_grouped.drop('Neighborhood', 1)
km = KMeans(n_clusters=kclusters, random_state=0).fit(dt_group_cluster)
km.labels_[0:10] 

array([0, 0, 2, 0, 3, 0, 5, 0, 3, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [32]:
neighborhoods_venues_sorted.insert(0, 'Labels', km.labels_)
dt_merged = trt
dt_merged = dt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

dt_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.6796,-79.3775,1,Park,Trail,Playground,Creperie,...,Cosmetics Shop,Eastern European Restaurant,Concert Hall,Comic Shop,Comfort Food Restaurant,Colombian Restaurant,College Rec Center,College Gym,College Cafeteria,College Auditorium
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668,-79.3677,0,Coffee Shop,Café,Restaurant,Pub,...,Beer Store,Convenience Store,Market,Liquor Store,Deli / Bodega,Diner,Jewelry Store,Japanese Restaurant,Indian Restaurant,Flower Shop
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,...,Grocery Store,Health & Beauty Service,Hobby Shop,Ice Cream Shop,Indian Restaurant,Juice Bar,Mexican Restaurant,Food & Drink Shop,Dog Run,Ethiopian Restaurant
3,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606,7,Coffee Shop,Bakery,Park,Café,...,Electronics Store,Bank,Performing Arts Venue,Chocolate Shop,Cosmetics Shop,Dessert Shop,Beer Store,Antique Shop,French Restaurant,Hotel
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3789,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,...,Diner,Bookstore,Electronics Store,Pizza Place,Middle Eastern Restaurant,Burger Joint,Mexican Restaurant,Burrito Place,Movie Theater,Office


Visualize the resulting clusters on map:

In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Latitude'], dt_merged['Longitude'], dt_merged['Neighbourhood'], dt_merged['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=9,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [34]:
dt_merged.loc[dt_merged['Labels'] == 0, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
1,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Pub,Italian Restaurant,Bakery,Pizza Place,Caribbean Restaurant,...,Beer Store,Convenience Store,Market,Liquor Store,Deli / Bodega,Diner,Jewelry Store,Japanese Restaurant,Indian Restaurant,Flower Shop
2,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Café,Pub,Men's Store,...,Grocery Store,Health & Beauty Service,Hobby Shop,Ice Cream Shop,Indian Restaurant,Juice Bar,Mexican Restaurant,Food & Drink Shop,Dog Run,Ethiopian Restaurant
4,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Tea Room,Lingerie Store,Italian Restaurant,...,Diner,Bookstore,Electronics Store,Pizza Place,Middle Eastern Restaurant,Burger Joint,Mexican Restaurant,Burrito Place,Movie Theater,Office
5,Downtown Toronto,0,Café,Restaurant,Coffee Shop,American Restaurant,Breakfast Spot,Clothing Store,Cocktail Bar,Beer Bar,...,Park,Lingerie Store,BBQ Joint,Farmers Market,Electronics Store,Furniture / Home Store,Food & Drink Shop,New American Restaurant,Office,Food Truck
6,Downtown Toronto,0,Coffee Shop,Beer Bar,Cheese Shop,Seafood Restaurant,Café,Farmers Market,Cocktail Bar,Steakhouse,...,Hotel,Restaurant,Bistro,Concert Hall,Breakfast Spot,Park,Diner,Italian Restaurant,Japanese Restaurant,Jazz Club
8,Downtown Toronto,0,Coffee Shop,Café,Thai Restaurant,Steakhouse,Burger Joint,Restaurant,Bar,Bakery,...,Pizza Place,Vegetarian / Vegan Restaurant,Gym,American Restaurant,Gastropub,New American Restaurant,Mediterranean Restaurant,Electronics Store,Clothing Store,Modern European Restaurant
13,Downtown Toronto,0,Bar,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Coffee Shop,Chinese Restaurant,Bakery,...,Grocery Store,Burrito Place,Breakfast Spot,Record Shop,Bubble Tea Shop,Hospital,Poutine Place,Jazz Club,Thai Restaurant,Japanese Restaurant


In [35]:
dt_merged.loc[dt_merged['Labels'] == 1, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,Downtown Toronto,1,Park,Trail,Playground,Creperie,Donut Shop,Doner Restaurant,Dog Run,Discount Store,...,Cosmetics Shop,Eastern European Restaurant,Concert Hall,Comic Shop,Comfort Food Restaurant,Colombian Restaurant,College Rec Center,College Gym,College Cafeteria,College Auditorium


In [30]:
#Cluster 3
dt_merged.loc[dt_merged['Labels'] == 2, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Airport Gate,Airport,Airport Food Court


In [36]:
dt_merged.loc[dt_merged['Labels'] == 3, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
7,Downtown Toronto,3,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Burger Joint,Juice Bar,Japanese Restaurant,Ice Cream Shop,...,Yoga Studio,Office,Miscellaneous Shop,Park,Poke Place,Portuguese Restaurant,Modern European Restaurant,Korean Restaurant,Metro Station,Comic Shop
9,Downtown Toronto,3,Coffee Shop,Aquarium,Café,Hotel,Italian Restaurant,Brewery,Sporting Goods Shop,Fried Chicken Joint,...,Bar,History Museum,Baseball Stadium,Hotel Bar,Chinese Restaurant,Juice Bar,Wine Bar,Lounge,Beer Bar,Dessert Shop
10,Downtown Toronto,3,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Bar,Seafood Restaurant,American Restaurant,...,Lounge,Tea Room,Gym / Fitness Center,Burger Joint,Pub,French Restaurant,Plaza,Pizza Place,Fried Chicken Joint,Bookstore
11,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Hotel,Gym,Italian Restaurant,Steakhouse,Deli / Bodega,...,Japanese Restaurant,Vegetarian / Vegan Restaurant,Concert Hall,Burger Joint,Breakfast Spot,Cocktail Bar,Museum,Latin American Restaurant,Park,Building
15,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Beer Bar,Japanese Restaurant,Seafood Restaurant,Italian Restaurant,Hotel,...,Breakfast Spot,Lounge,Cheese Shop,Hostel,French Restaurant,Pharmacy,Diner,Poke Place,Pizza Place,Deli / Bodega
16,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Steakhouse,Japanese Restaurant,Seafood Restaurant,Asian Restaurant,Bar,...,American Restaurant,Italian Restaurant,Beer Bar,Concert Hall,Pizza Place,Bakery,Opera House,Noodle House,Museum,Building


Conclusion:

1. Cluster 1: the most common venue type is Coffee Shop, followed by restaurants and bars.
2. Cluster 2: the most common venue type is Park or Playground.
3. Cluster 3: the most common venue type is Airport Lounge.
4. Cluster 4: the most common venue type is Grocery Store.
